In [105]:
import torch                #导入torch库，里面包含了关于张量的各种运算
import torchvision          #导入torchvision库，主要用于处理图像数据。里面包含了常用的数据集（如MINIST、COCO）、经典网络结构、各种关于图像处理的函数（如归一化）
from torch.utils.data import DataLoader   
from torch.utils.tensorboard import SummaryWriter   #tensorboard使模型训练可视化，可以看到训练过程中loss的变化曲线；summarywrite可以将训练日志写入指定文件夹下的事件文件
from torch.utils import data
device = "cuda"                 #cuda为GPU编程接口，在这里device是作为tensor或者model被分配到的位置                                                                                                                                                             

In [106]:
trainDataLoard = torch.utils.data.DataLoader(  #定义一个装载可迭代的训练数据集的容器，以下为容器中的数据以及对其的操作 
    torchvision.datasets.MNIST('./data/', train=True, download=True,    #确定MNIST数据集将要放置的位置, train=True表示下载的是训练集, train=False表示下载的是测试集
                               transform=torchvision.transforms.Compose([   #transforms是对图像进行预处理的函数集，其中的compose函数作用是：将各函数联合起来发挥作用
                                   torchvision.transforms.ToTensor(),       #ToTensor的作用是将此数据集转换成张量
                                   torchvision.transforms.Normalize((0.456), (0.224)),  #Normalize(mean, std)为归一化函数，mean参数是数据集的平均值，std参数是数据集的标准差。通过输入这两个参数，便可以将数据归一化到我们想要的效果
                               ])),
    batch_size=2048, shuffle=True)      #batch_size为进行一次迭代的数据规模。shuffle()函数的作用是将数据的顺序打乱
testDataLoard = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/', train=False, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize((0.456), (0.224))
                               ])),
    batch_size=1, shuffle=True)
#以上为对数据的处理，接下来要进行网络结构的构造

In [107]:
class zyn(torch.nn.Module):
    def __init__(self):
        super(zyn, self).__init__()
        self.conv0 = torch.nn.Conv2d(1, 10, (3, 3), 1, 1)           
        self.conv1 = torch.nn.Conv2d(10, 10, (3, 3), 1, 1)
        self.conv2 = torch.nn.Conv2d(10, 10, (3, 3), 1, 1)
        self.conv3 = torch.nn.Conv2d(10, 10, (3, 3), 1, 1)
        self.conv4 = torch.nn.Conv2d(10, 10, (3, 3), 1, 1)
        self.conv5 = torch.nn.Conv2d(10, 10, (3, 3), 1, 1)
        self.conv6 = torch.nn.Conv2d(10, 10, (3, 3), 1, 1)
        self.conv7 = torch.nn.Conv2d(10, 10, (3, 3), 1, 1)
        self.conv8 = torch.nn.Conv2d(10, 10, (3, 3), 1, 1)
        '''
        现在开始构建网络结构。
        conv2d表示该卷积层处理的数据是二维的。
        nn.Conv2d中的参数介绍:
        第一位表示输入层神经元的个数,第二位为输出层神经元的个数,第三位为卷积核的size,第四位为卷积核的步长,第五位为padding,默认为0,第六位为padding_mode(填充方式,默认为常数填充)
        这里一共有8层卷积层,原始数据输入层不算作神经网络的层数
        '''                 
        self.f = torch.nn.Flatten()    #Flatten(dim)的作用是降维，dim表示从第dim个维度展开，将后面的维度降为一维。这个函数常用在卷积层到全连接层的过度。
        self.connected = torch.nn.Linear(7840, 10, True) 
        '''
        Linear为全连接层,一般放在卷积层的后面。这一层相当于将之前提取出的特征值组合起来。
        各参数的含义:
        第一个为输入神经元的个数。由于全连接之前的flatten函数已经将数据展开了,所以输入神经元的个数为(28 * 28 * 10),
        28 * 28是图像的大小,因为是灰度图,所以还有个乘1省略了,10则表示铺平前该层的神经元个数.
        需要注意的是, 由于我选择填充了一层, 所以经过三次卷积后图片尺寸未发生改变.
        若不选择填充, 这里填入的应该是(22 * 22 * 10).
        第二个参数表示输出神经元的个数,因为该程序是一个十分类的问题(0 ~ 9),所以输出神经元的个数应为10.
        第三个参数表示bias(偏置参数,起到微调的作用),True表示希望添加这样一个偏置值,否则填False.

        '''
        self.relu = torch.nn.ReLU(True)     #True表示希望ReLu对输出的数据进行改变
        # 定义向前传播的过程
    def forward(self, X):
        X = self.conv0(X)  
        X = self.conv1(X)
        X = self.conv2(X)
        X = self.relu(X)
        X = self.f(X)
        X = self.connected(X)  
        X = self.relu(X)             #向前传播的过程
        return X

        

        

In [108]:
net = zyn()         #实例化上面定义的网络结构
net.to(device)      #指定网络结构运行的地方
opt = torch.optim.Adam(net.parameters(), 0.002)  
'''
optim意为最好的。这里使用Adam算法对网络进行优化。
Adam里面的参数介绍:
第一个参数为params(翻译成中文也是参数的意思),这里填入net.parameters()相当于填入了一个优化器,将返回来的参数进行优化后再将其填入params的位置.
第二个参数是lr(learn rate),默认为0.001,学习率过大可能会出现梯度爆炸从而错过最低点,过小可能会出现梯度消失的现象,导致学习的速率很慢.
第三个参数是betas = (0.9, 0.999),通常被用于正则化,防止模型过拟合.而正则化的作用为减少不那么重要的特征变量的数量. 所以beta用于调整L1或L2正则化项的权重.
第四个参数是eps,默认值为1e-8,是加到分母里面的项,用于防止计算时出现除以0的错误,保证数值的稳定性.
第五个参数是weight_decay(默认为0), 权重衰减, 是L2正则化项的一个系数.

'''
loss = torch.nn.CrossEntropyLoss()  #交叉熵损失函数，常用于多分类的网络模型中。用于衡量实际输出与预期输出的距离。这里相当于给原本的函数换个名字。
loss.to(device)         #指定损失函数运行的地方

CrossEntropyLoss()

In [109]:
#迭代开始
for epoch in range(25):
    '''
    epoch为全部数据输入后进行了一次向前传播和向后传播的过程.
    iteration(迭代)为batch_size的batch进行一次向前传播和向后传播的过程.
    所以假如一共有1000个数据, batch_size = 100, 那么epoch = 10(iteration).
    如果数据不能整除batch_size, 那么可以自己设置是否要将除不断的部分丢弃不进行迭代.
    比如:一共有1007个数据, batch_size = 100, 那么还剩7个, 如果丢弃, epoch = 10(iteration), 否则epoch = 11(iteration).
    在这里, 我将全部数据的迭代次数设置为25次.
    '''
    for image, label in trainDataLoard:     #对所有数据进行一次遍历
        opt.zero_grad()                  #zero_grad()的作用是将梯度清零. 因为各个mini_batch之间的梯度并不需要混合运算
        image = image.to(device)            #将image和label复制到设定的device上运算
        label = label.to(device)
        label = torch.nn.functional.one_hot(label).float()
        '''
        将label转换成one-hot编码.
        one-hot函数的参数介绍:
        第一个参数是需要转换的标签, 在本程序中做的是手写数字识别, 所以标签为 0 ~ 9 这10个数字.(注意填入的是张量).
        第二个参数是总的类别数(相当于one-hot编码的列数), 在这个程序中一用有10个数字需要识别, 所以类别数为10.(注意, 这个参数是int型).
        举个简单的例子:
        假如我们要识别的是("cat", "dog", "pig"), 那么num_class=3, 转化成one-hot编码为以下结果:
        tensor(
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
        ),
        如果我们没有指定num_class的值, 那么one-hot的列数为标签中的(max + 1).(因为从0到max一共有(max + 1)个数)
        比如, 我们输入的标签是tensor([1, 2]), 并且没有指定num_class, 那么转换成one-hot编码后将得到如下结果:
        tensor(
        [0, 1, 0],
        [0, 0, 1],
        )
        事实上, 在我们下载的数据集里面已经包含了各标签, 所以并不需要我们手动设置需要转换的类别和类别数, 否则可能会有维度不匹配的报错.
        '''
        out = net(image)        #这里就将数据输入进了之前定义好的网络结构中了, 并将输出的数据赋给变量out.
        Loss = loss(out, label)
        '''
        上一个语句中数据已经向前传播过一次了, 所以这里计算输入数据的损失函数.
        loss函数(前面已经提到这是torch.nn.CrossEntropyLoss()函数的别名)的参数介绍;
        第一个参数是实际输出的数据, 第二个参数是预期得到的数据.
        这里需要注意的是, 根据定义, loss函数是指单个数据的实际输出与预期数据之间的距离, cost函数是指一组数据的实际输出与预期数据的距离.(就是将该组数据中各loss函数相加, 然后求平均值)
        而在这里的loss函数, 其实已经将mini_batch中所有的损失函数相加求了平均值并赋值给了我们定义的变量, 所以这里的loss函数其实是相当于mini_batch的成本函数.
        其实在torch.nn.CrossEntropyLoss()中有一个默认参数为reduction = 'mean'(将各损失函数相加求平均值), 我们也可以手动填入reduction = 'sum', 这样输出的便是损失函数的和.
        并且, 交叉熵损失函数里面包含了softmax(), 所以在设置向前传播的过程中, 可以不加上这个softmax函数.
        '''
        print(float(Loss))
        Loss.backward()     #进行向后传播
        opt.step()          #更新权重
        #训练集训练结束
torch.save(zyn, 'net.pkl')

2.2966256141662598
2.1384899616241455
1.8355185985565186
1.5128458738327026
1.1157293319702148
0.7949536442756653
0.5668877363204956
0.4767947793006897
0.4585520625114441
0.4241270124912262
0.3954535126686096
0.4243759512901306
0.42235198616981506
0.31705835461616516
0.32847267389297485
0.27531400322914124
0.3032604455947876
0.31686216592788696
0.31460291147232056
0.2783046364784241
0.2599778175354004
0.2685944437980652
0.21238401532173157
0.21999524533748627
0.2302144169807434
0.22167041897773743
0.19857844710350037
0.21444308757781982
0.21293413639068604
0.205055832862854
0.16719050705432892
0.1965045928955078
0.16155767440795898
0.19544990360736847
0.15338554978370667
0.15163710713386536
0.11737067997455597
0.16304028034210205
0.1324148178100586
0.1339157521724701
0.1451118290424347
0.1459941267967224
0.12013877928256989
0.14679329097270966
0.13150504231452942
0.1375003457069397
0.1331150233745575
0.13572552800178528
0.13917915523052216
0.10383671522140503
0.12125859409570694
0.1218

In [110]:
all = 0
corr = 0
w = SummaryWriter()
for image, label in testDataLoard:
    image = image.to(device)
    out = net(image)
    out1 = out.data.max(1, keepdim=True)[1]         #将概率值最大的数赋给out1
    if int(label) == int(out1):
        corr += 1
    else:
        #w.add_image(str(out1), image[0])
        print(label, ' ', out1)     #打印出识别错误的图片
    all += 1
w.close()
print("all: ", all, "\tcorr: ", corr)       

tensor([5])   tensor([[3]], device='cuda:0')
tensor([2])   tensor([[7]], device='cuda:0')
tensor([3])   tensor([[7]], device='cuda:0')
tensor([3])   tensor([[9]], device='cuda:0')
tensor([8])   tensor([[3]], device='cuda:0')
tensor([3])   tensor([[5]], device='cuda:0')
tensor([1])   tensor([[3]], device='cuda:0')
tensor([4])   tensor([[9]], device='cuda:0')
tensor([6])   tensor([[5]], device='cuda:0')
tensor([9])   tensor([[3]], device='cuda:0')
tensor([0])   tensor([[8]], device='cuda:0')
tensor([5])   tensor([[3]], device='cuda:0')
tensor([2])   tensor([[0]], device='cuda:0')
tensor([6])   tensor([[0]], device='cuda:0')
tensor([2])   tensor([[1]], device='cuda:0')
tensor([2])   tensor([[1]], device='cuda:0')
tensor([5])   tensor([[3]], device='cuda:0')
tensor([3])   tensor([[5]], device='cuda:0')
tensor([9])   tensor([[4]], device='cuda:0')
tensor([4])   tensor([[2]], device='cuda:0')
tensor([2])   tensor([[4]], device='cuda:0')
tensor([5])   tensor([[0]], device='cuda:0')
tensor([9]